In [1]:
from bs4 import BeautifulSoup
import urllib
import re
import json

**The scripts for the second season are in different formats.**

This section handles parsing of Season 1, episodes 1-5 and all of Season 2.

In [2]:
lynchsite = urllib.urlopen('http://www.glastonberrygrove.net/texts/main.html').read()
lynchsite_soup = BeautifulSoup(lynchsite, 'html.parser')

In [3]:
script_links=['http://www.lynchnet.com/tp/tp01.html', 'http://www.lynchnet.com/tp/tp02.html', 'http://www.lynchnet.com/tp/tp03.html', 'http://www.lynchnet.com/tp/tp04.html', 'http://www.lynchnet.com/tp/tp05.html']
hand_script_links=[]
for link in lynchsite_soup.find_all('a'):
     if re.search('script\d',str(link.get('href'))) is not None:
            script_links.append('http://www.glastonberrygrove.net/texts/'+str(link.get('href')))

script_links

['http://www.lynchnet.com/tp/tp01.html',
 'http://www.lynchnet.com/tp/tp02.html',
 'http://www.lynchnet.com/tp/tp03.html',
 'http://www.lynchnet.com/tp/tp04.html',
 'http://www.lynchnet.com/tp/tp05.html',
 'http://www.glastonberrygrove.net/texts/script8.html',
 'http://www.glastonberrygrove.net/texts/script9.html',
 'http://www.glastonberrygrove.net/texts/script10.html',
 'http://www.glastonberrygrove.net/texts/script11.html',
 'http://www.glastonberrygrove.net/texts/script12.html',
 'http://www.glastonberrygrove.net/texts/script13.html',
 'http://www.glastonberrygrove.net/texts/script14.html',
 'http://www.glastonberrygrove.net/texts/script15.html',
 'http://www.glastonberrygrove.net/texts/script16.html',
 'http://www.glastonberrygrove.net/texts/script17.html',
 'http://www.glastonberrygrove.net/texts/script18.html',
 'http://www.glastonberrygrove.net/texts/script19.html',
 'http://www.glastonberrygrove.net/texts/script20.html',
 'http://www.glastonberrygrove.net/texts/script21.html',

In [4]:
###FIRST SCRIPT. SECOND ONE WORKED BETTER WITH SPACE ISSUES
def script_script(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, 'html.parser')
    script = soup.get_text()
    #gets all the scenes and characters
    scenes_chars = re.findall('\n(\d.+|[A-Z ]+)\n', script)
    print scenes_chars
    scene = 0
    scene_dict = {}
    #parses the script into a dictionary
    for line in scenes_chars:
        scene_num = re.search('(\d+\w*)[\.\: ]', line)
        #establishes the scene and all its info
        if scene_num is not None:
            #print line
            scene = scene_num.group(1)
            scene_dict[scene] = {'scene_number': scene, 'characters':[], 'heading':{}}
            #inside or outside
            if "INT" in line:
                scene_dict[scene]['heading']['int_ext'] = 'INT'
            elif "EXT" in line:
                scene_dict[scene]['heading']['int_ext'] = 'EXT'
            #grabs location
            location = re.search('[A-Z]\. (.*) \-',line)
            if location is not None:
                scene_dict[scene]['heading']['location'] = location.group(1)
            #gets time
            time = re.search('- ([A-Z]+)$',line)
            if time is not None:
                scene_dict[scene]['heading']['time'] = time.group(1)
        elif scene == 0:
            pass
        else:
            if 'FADE' not in line and 'ACT' not in line and 'SCENE' not in line:
                if line not in scene_dict[scene]['characters']:
                    scene_dict[scene]['characters'].append(line)
    
    return scene_dict


In [9]:
def script_script2(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, 'html.parser')
    script = soup.get_text()
    #gets all the scenes and characters
    scenes_chars = re.findall('\n\s*(\d.+|[A-Z ]+\s*)\n\s*', script)
    for a in scenes_chars:
        print a
    scene = 0
    scene_dict = {}
    #parses the script into a dictionary
    for line in scenes_chars:
        scene_num = re.search('(\d+\w*)[\.\: ]', line)
        #establishes the scene and all its info
        if scene_num is not None and scene_num not in scene_dict:
            #print line
            scene = scene_num.group(1)
            scene_dict[scene] = {'scene_number': scene, 'characters':[], 'heading':{}}
            #inside or outside
            if "INT" in line:
                scene_dict[scene]['heading']['int_ext'] = 'INT'
            elif "EXT" in line:
                scene_dict[scene]['heading']['int_ext'] = 'EXT'
            #grabs location
            location = re.search('[A-Z]\.\s+(.*)\s+\-',line)
            if location is not None:
                scene_dict[scene]['heading']['location'] = location.group(1)
            #gets time
            time = re.search('- ([A-Z]+)$',line)
            if time is not None:
                scene_dict[scene]['heading']['time'] = time.group(1)
        elif scene == 0:
            pass
        else:
            if 'FADE' not in line and 'ACT' not in line and 'SCENE' not in line:
                if line not in scene_dict[scene]['characters'] and line !=" " and "\n" not in line:
                    scene_dict[scene]['characters'].append(line)
    
    return scene_dict


In [10]:
episode_dict = {}
for link in script_links:
    episode_dict[int(re.search('(\d*).html',link).group(1))] = script_script2(link)
#print episode_dict
outfile = open('twin_peaks.json', 'w')
json.dump(episode_dict, outfile, indent=4)
outfile.close()
    

TWIN PEAKS
MARK FROST DAVID LYNCH











 
ACT ONE
1. EXT. GREAT NORTHERN HOTEL - DAY
2. INT. GREAT NORTHERN HOTEL ROOM - DAY
COOPER
3. EXT. SHERIFF TRUMAN'S HOUSE - MORNING
TRUMAN
 
4. EXT. DOUGHNUT SHOP - DAY
5. INT. DOUGHNUT SHOP - MORNING
 
LUCY
ANDY
6. INT. SHERIFF'S CRUISER - MORNING
 
TRUMAN
LUCY
 
7. INT. GREAT NORTHERN HOTEL DINING ROOM - DAY
COOPER
WAITRESS
COOPER
 
8. AUDREY HORNE
 
 
AUDREY
COOPER
AUDREY
COOPER
 
AUDREY
COOPER
AUDREY
COOPER
AUDREY
COOPER
AUDREY
COOPER
AUDREY
 
9. BENJAMIN HORNE
 
10. EXT. LEO JOHNSON'S HOUSE MORNING
SHELLY
 
LEO
SHELLY
LEO
SHELLY
LEO
 
SHELLY
LEO
SHELLY
 
11.  INT. JOHNSON KITCHEN - MORNING
 
 
 
 
LEO
SHELLY
 
LEO
 
SHELLY
 
 
12. EXT. JOHNSON HOUSE/INT. NORMA'S CAR - MORNING
NORMA
SHELLY
NORMA
SHELLY
 
SHELLY
NORMA
SHELLY
NORMA
SHELLY
 
NORMA
 

SHELLY
NORMA
 
13. EXT. SHERIFF'S STATION ENTRANCE - MORNING
14. INT. SHERIFFS STATION RECEPTION AREA - MORNING
COOPER
ANDY
 
COOPER
LUCY
COOPER
 
15. INT. INTERROGATION ROOM A - MORNING
 

CO

Idea.... if I have a scene with no discussion, look for characters names?

Below this doesn't work. Still missing data for Pilot, episode 6 and 7.

In [7]:
def script_script3(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, 'html.parser')
    script = soup.get_text()
    #gets all the scenes and characters
    scenes_chars = re.findall('\n*\s*(\d.+|[A-Z]+\s*)\n+\s*', script)
    print scenes_chars
    scene = 0
    scene_dict = {}
    #parses the script into a dictionary
    for line in scenes_chars:
        print line
        #scene_num = re.search('(\d+\w*)[\.\: ]', line)
        #establishes the scene and all its info
        if 'EXT' in line or 'INT' in line:
            
            scene +=1
            scene_dict[scene] = {'scene_number': scene, 'characters':[], 'heading':{}}
            #inside or outside
            if "INT" in line:
                scene_dict[scene]['heading']['int_ext'] = 'INT'
            elif "EXT" in line:
                scene_dict[scene]['heading']['int_ext'] = 'EXT'
            #grabs location
            location = re.search('[A-Z]\.\s+(.*)\s+\-',line)
            if location is not None:
                scene_dict[scene]['heading']['location'] = location.group(1)
            #gets time
            time = re.search('- ([A-Z]+)$',line)
            if time is not None:
                scene_dict[scene]['heading']['time'] = time.group(1)
                
        elif scene == 0:
            pass
        else:
            if 'FADE' not in line and 'ACT' not in line and 'SCENE' not in line:
                if line not in scene_dict[scene]['characters']:
                    scene_dict[scene]['characters'].append(line)
    
    print scene_dict
    return scene_dict

In [8]:
script_script2('http://www.glastonberrygrove.net/texts/script8.html')

[u'TWIN PEAKS', u' \n\n\n\n\n\n', u'7700 Balboa Blvd', u'ACT ONE\n', u'1. EXT. GREAT NORTHERN HOTEL - NIGHT', u'2. INT. GREAT NORTHERN CORRIDOR - NIGHT', u'WAITER', u'COOPER', u'WAITER', u'WAITER', u'COOPER', u'WAITER', u'COOPER', u'WAITER', u'COOPER', u'WAITER', u'WAITER', u'2.001', u'GIANT', u'COOPER', u'GIANT', u'COOPER', u'GIANT', u'COOPER', u'GIANT', u'COOPER', u'GIANT', u'COOPER', u'GIANT', u'COOPER', u"3. EXT. ONE-EYED JACK'S - NIGHT", u"4. INT. ONE-EYED JACK'S BEDROOM - NIGHT", u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u'BEN', u'BEN', u'AUDREY', u'BEN', u'AUDREY', u'BEN', u"5. INT. BLACKIE'S OFFICE - NIGHT", u'BEN', u'JERRY', u'BEN', u'JERRY', u'BEN', u'JERRY', u'BEN', u'JERRY', u'BEN', u'JERRY', u'BEN', u'BLACKIE', u'BEN', u'BLACK

{u'1': {'characters': [],
  'heading': {'int_ext': 'EXT',
   'location': u'GREAT NORTHERN HOTEL',
   'time': u'NIGHT'},
  'scene_number': u'1'},
 u'10': {'characters': [],
  'heading': {'int_ext': 'EXT',
   'location': u'CALHOUN MEMORIAL HOSPITAL',
   'time': u'DAY'},
  'scene_number': u'10'},
 u'11': {'characters': [],
  'heading': {'int_ext': 'INT',
   'location': u'CALHOUN HOSPITAL INTENSIVE CARE',
   'time': u'DAY'},
  'scene_number': u'11'},
 u'12': {'characters': [u'COOPER', u'HAYWARD', u'TRUMAN', u'LUCY'],
  'heading': {'int_ext': 'INT'},
  'scene_number': u'12'},
 u'13': {'characters': [],
  'heading': {'int_ext': 'INT', 'location': u'HOSPITAL ROOM', 'time': u'DAY'},
  'scene_number': u'13'},
 u'14': {'characters': [u'TRUMAN', u'SHELLY'],
  'heading': {'int_ext': 'INT', 'location': u'HOSPITAL ROOM', 'time': u'DAY'},
  'scene_number': u'14'},
 u'15': {'characters': [u'TRUMAN', u'ED', u'HAYWARD', u'COOPER'],
  'heading': {'int_ext': 'INT',
   'location': u'HOSPITAL CORRIDOR',
   